## Build a New Agent Application

<table align="left">
    <td style="text-align: center">
        <a href="https://colab.research.google.com/github/ssvaidyanathan/apigee-samples/blob/main/llm-vertexai-agent/llm_vertexai_agent.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo\"><br> Open in Colab
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fssvaidyanathan%2Fapigee-samples%2Fmain%2Fllm-vertexai-agent%2Fllm_vertexai_agent.ipynb">
          <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
        </a>
      </td>    
      <td style="text-align: center">
        <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/ssvaidyanathan/apigee-samples/main/llm-vertexai-agent/llm_vertexai_agent.ipynb">
          <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
        </a>
      </td>
      <td style="text-align: center">
        <a href="https://github.com/ssvaidyanathan/apigee-samples/blob/main/llm-vertexai-agent/llm_vertexai_agent.ipynb">
          <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
        </a>
      </td>
</table>
<br />
<br />
<br />

## Setup

Use the following GCP CloudShell tutorial. Follow the instructions to deploy the sample.

[![Open in Cloud Shell](https://gstatic.com/cloudssh/images/open-btn.png)](https://ssh.cloud.google.com/cloudshell/open?cloudshell_git_repo=https://github.com/ssvaidyanathan/apigee-samples&cloudshell_git_branch=main&cloudshell_workspace=.&cloudshell_tutorial=llm-vertexai-agent/docs/cloudshell-tutorial.md)

In [ ]:
#If you haven't already, make sure you install the `dfcx-scrapi` library
!pip install dfcx-scrapi

import sys

# Colab Auth needed to call Client Endpoints (i.e. vertexai)
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

### Initialize Variables

In [3]:
PROJECT_ID = "" # @param {type:"string"}
REGION = "us-east1" # @param {type:"string"}
APIGEE_HOST = "" # @param {type:"string"}
AGENT_NAME = "Apigee AI AGENT"

## Create the agent

In [ ]:
from dfcx_scrapi.core.agents import Agents
from dfcx_scrapi.core.sessions import Sessions

a = Agents()
agent = a.create_agent(
    project_id=PROJECT_ID,
    display_name=AGENT_NAME,
    gcp_region=REGION,
    playbook_agent=True
)

panel = "(playbooks/00000000-0000-0000-0000-000000000000/basics//right-panel:simulator)"
print(f"AGENT LINK: https://vertexaiconversation.cloud.google.com/{agent.name}/{panel}")

s = Sessions()

session_id = s.build_session_id(agent.name)
session_id

### Check for default playbook

In [ ]:
from dfcx_scrapi.core.playbooks import Playbooks

p = Playbooks(agent.name)

playbooks_map = p.get_playbooks_map(agent.name, reverse=True)
playbook = p.get_playbook(playbooks_map["Default Generative Playbook"])
print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")


## Create Tool

In [6]:
get_products = f"""
openapi: 3.0.1
info:
  title: Cymbal Products Catalog
  description: Product service

  termsOfService: https://cymbal.com/terms
  contact:
    email: someteam@cymbal.com
  license:
    name: Apache 2.0
    url: http://www.apache.org/licenses/LICENSE-2.0.html
  version: 1.0.0
servers:
  - url: https://{APIGEE_HOSTNAME}/v1/samples/llm-vertexai-agent
tags:
  - name: products
    description: Products
paths:
  '/products':
    get:
      tags:
        - products
      summary: Get Products
      description: Get Products
      operationId: get-all-products
      responses:
        '200':
          description: successful operation
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Products'
        '400':
          description: Invalid request supplied
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '401':
          description: Unauthorized
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '403':
          description: Forbidden
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '404':
          description: Not Found
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '429':
          description: Not Found
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
      #security:
        #- api_key: []
  '/products/{{id}}':
    get:
      tags:
        - products
      summary: Get a Product
      description: Get a Product
      operationId: get-product
      parameters:
        - name: id
          in: path
          description: ProductId
          required: true
          schema:
            type: integer
            example: 1
      responses:
        '200':
          description: successful operation
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Products'
        '400':
          description: Invalid request supplied
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '401':
          description: Unauthorized
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '403':
          description: Forbidden
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '404':
          description: Not Found
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
        '429':
          description: Not Found
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ErrorResponse'
      #security:
        #- api_key: []
components:
  schemas:
    Products:
      type: object
      properties:
        products:
          type: array
          items:
            $ref: '#/components/schemas/Product'
    Product:
      type: object
      properties:
        id:
          type: integer
          example: 1
        name:
          type: string
          example: Daily Sunglasses
        description:
          type: string
          example: "Seriously coolMen's Daily Sunglasses for only $19.90"
        categories:
          type: array
          items:
            type: string
            example: "Style"
        price:
          type: number
          example: 19
    ErrorResponse:
      type: object
      properties:
        status:
          type: string
          example: Forbidden
        message:
          type: string
          example: Forbidden
  #securitySchemes:
    #api_key:
      #type: apiKey
      #name: x-apikey
      #in: header
"""

In [7]:
from dfcx_scrapi.core.tools import Tools

t = Tools()

# get_products
get_products_tool = t.build_open_api_tool(
    display_name = "cymbal-products-catalog",
    spec = get_products,
    description = "Cymbal Products Catalog",
    )
products_tool = t.create_tool(agent.name, get_products_tool)

## Update playbook

In [ ]:
playbook = p.update_playbook(
    playbooks_map["Default Generative Playbook"],
    referenced_tools=[products_tool.name],
    goal="You are a friendly online boutique  service center agent.\nYour job is to answer any product related questions",
    instructions=["Greet the users, then ask how you can help them today.",
                  "If the user requests help looking for something, query the ${TOOL:cymbal-products-catalog} to answer the product questions",
                  "Also provide answers for general knowledge questions"]
    )

print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")

## Test the Agent

Lets ask the agent some questions about the product to see if it is able to fetch the information from the API

In [ ]:
session_id = s.build_session_id(agent.name)

conversation = [
    "List all products",
    "tell me more about the sunglasses."
    "how much is the mug?",
    "What are the product categories?",
    "what is the interior of the mug?"]

i = 1
for utterance in conversation:
  print(f"\n--- TURN {i} --- \n")
  res = s.detect_intent(agent.name, session_id, utterance)
  s.parse_result(res)
  i+=1